### LSTM in PyTorch for Character Generation

#on top of the code by LeanManager for my personal understanding. Original copyright for LSTM belongs to Hochreiter & Schmidhuber. 

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [19]:
# open text file and read in data as `text`

#with open('anna.txt', 'r') as f:
with open('anna.txt', 'r') as f:

    text = f.read()


'''set("some_string")'''
#creates a dictionary of unique values from the items of the iterables passed to it.
#text = "Aashish"
#print(set(text)) prints {'s', 'A', 'h', 'i'}

#print(set(text))

'''print(tuple(set(text)))'''
#converts the dictionary into a comma-separated tuple



'print(tuple(set(text)))'

In [20]:
# encode the text and map each character to an integer and vice versa

# we create two dictonaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers



character_vocabulary = tuple(set(text))

int2char = dict(enumerate(character_vocabulary))
# creates {0: ')', 1: 'y', 2: 'a', 3: '/', 4: 'B', 5: 'f', 6: 'U', 7: 'X', .......and so on }

#just reverse the above key-value pairs 
char2int = {ch: ii for ii, ch in int2char.items()}

#convert the whole text into the corresponding integer value
each_char_in_encodeed_integer_form = np.array([char2int[ch] for ch in text])


In [21]:
def one_hot_encoder(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `no_of_seqs_per_batch` (also refered to as "batch size" in other places). Each of those sequences will be `no_of_timesteps_per_sequence` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ sequences. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `no_of_timesteps_per_sequence`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. The way I like to do this window is use `range` to take steps of size `no_of_timesteps_per_sequence` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `no_of_timesteps_per_sequence` wide.

In [22]:
def get_batches(array_of_characters, no_of_seqs_per_batch, no_of_timesteps_per_sequence):# _, 10, 50
    '''Create a generator that returns batches of size
       no_of_seqs_per_batch x no_of_timesteps_per_sequence from array_of_characters.
       
       Arguments
       ---------
       array_of_characters: Array you want to make batches from
       no_of_seqs_per_batch: Batch size, the number of sequences per batch
       no_of_timesteps_per_sequence: Number of sequence steps per batch
    '''
    
    
    batch_size = no_of_seqs_per_batch * no_of_timesteps_per_sequence
    no_of_batches = len(array_of_characters)//batch_size
    
    # Keep only enough characters to make full batches
    array_of_characters = array_of_characters[:no_of_batches * batch_size]
    
    # Reshape into no_of_seqs_per_batch rows (In our example, 2)
    array_of_characters = array_of_characters.reshape((no_of_seqs_per_batch, -1))
       
    for n in range(0, array_of_characters.shape[1], no_of_timesteps_per_sequence): #simple way to get the batches
        
        # The features
        x = array_of_characters[:, n:n+no_of_timesteps_per_sequence]
        
        # The targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], array_of_characters[:, n+no_of_timesteps_per_sequence] #sweet
#             print(x)
#             print(y)
            
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], array_of_characters[:, 0]
        yield x, y

In [23]:
class LSTM_Char_Generation(nn.Module):
    
    def __init__(self, unique_vocabulary_tokens, no_of_timesteps_per_sequence=100, no_of_hidden_nodes_in_each_lstm_cell=256, no_of_stacked_lstm_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.no_of_stacked_lstm_layers = no_of_stacked_lstm_layers
        self.no_of_hidden_nodes_in_each_lstm_cell = no_of_hidden_nodes_in_each_lstm_cell
        self.lr = lr
        
        # Creating character dictionaries
        self.character_vocabulary = unique_vocabulary_tokens
        
        self.int2char = dict(enumerate(self.character_vocabulary))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## Define the LSTM
        self.lstm = nn.LSTM(len(self.character_vocabulary), no_of_hidden_nodes_in_each_lstm_cell, no_of_stacked_lstm_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## Define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## Define the final, fully-connected output layer
        self.fc = nn.Linear(no_of_hidden_nodes_in_each_lstm_cell, len(self.character_vocabulary))
        
        # Initialize the weights
        self.init_weights()
      
    
    def forward(self, x, hidden_and_cell_states):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden and cell states `hidden_and_cell_states`. '''
        
        #Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hidden_and_cell_states)
        
        ## Pass x through the dropout layer
        x = self.dropout(x)
        
        # Stack up LSTM outputs using view
        x = x.view(x.size()[0]*x.size()[1], self.no_of_hidden_nodes_in_each_lstm_cell)
        
        ## Put x through the fully-connected layer
        x = self.fc(x)
        
        # Return x and the hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        
        #The output of our RNN is from a fully-connected layer and it outputs a distribution of next-character 
        #scores. To actually get the next character, we apply a softmax function, 
        #which gives us a probability distribution that we can then sample to predict the next character
        
        
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1) 
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encoder(x, len(self.character_vocabulary))
        
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.character_vocabulary))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, no_of_seqs_per_batch):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x no_of_seqs_per_batch x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.no_of_stacked_lstm_layers, no_of_seqs_per_batch, self.no_of_hidden_nodes_in_each_lstm_cell).zero_(),
                weight.new(self.no_of_stacked_lstm_layers, no_of_seqs_per_batch, self.no_of_hidden_nodes_in_each_lstm_cell).zero_())
        

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

In [24]:
def train(network, data, epochs=10, no_of_seqs_per_batch=10, no_of_timesteps_per_sequence=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        no_of_seqs_per_batch: Number of mini-sequences per mini-batch, aka batch size
        no_of_timesteps_per_sequence: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    
    '''
    
    network.train()
    
    opt = torch.optim.Adam(network.parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
        network.cuda()
    
    counter = 0
    n_chars = len(network.character_vocabulary)
    
    for e in range(epochs):
        
        h = network.init_hidden(no_of_seqs_per_batch)
        
        for x, y in get_batches(data, no_of_seqs_per_batch, no_of_timesteps_per_sequence):
            
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encoder(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            network.zero_grad()
            
            output, h = network.forward(inputs, h)
            
            loss = criterion(output, targets.view(no_of_seqs_per_batch*no_of_timesteps_per_sequence).type(torch.cuda.LongTensor))

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(network.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = network.init_hidden(no_of_seqs_per_batch)
                val_losses = []
                
                for x, y in get_batches(val_data, no_of_seqs_per_batch, no_of_timesteps_per_sequence):
                    
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encoder(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = network.forward(inputs, val_h)
                    val_loss = criterion(output, targets.view(no_of_seqs_per_batch*no_of_timesteps_per_sequence).type(torch.cuda.LongTensor))
                
                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [25]:

if 'network' in locals(): # function returns a dictionary containing the variables defined in the local namespace
    del network

In [26]:
# Initialize the network

network = LSTM_Char_Generation(character_vocabulary, no_of_hidden_nodes_in_each_lstm_cell=512, no_of_stacked_lstm_layers=2)
print(network)

LSTM_Char_Generation(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [31]:
no_of_seqs_per_batch, no_of_timesteps_per_sequence = 128, 100
#set to training mode
network.train()
train(network, each_char_in_encodeed_integer_form, epochs= 40, no_of_seqs_per_batch=no_of_seqs_per_batch, no_of_timesteps_per_sequence=no_of_timesteps_per_sequence, lr=0.0001, cuda=True, print_every=10)

Epoch: 1/40... Step: 10... Loss: 1.8579... Val Loss: 1.9519
Epoch: 1/40... Step: 20... Loss: 1.8599... Val Loss: 1.9406
Epoch: 1/40... Step: 30... Loss: 1.8395... Val Loss: 1.9417
Epoch: 1/40... Step: 40... Loss: 1.8270... Val Loss: 1.9353
Epoch: 1/40... Step: 50... Loss: 1.8468... Val Loss: 1.9408
Epoch: 1/40... Step: 60... Loss: 1.7811... Val Loss: 1.9303
Epoch: 1/40... Step: 70... Loss: 1.7993... Val Loss: 1.9244
Epoch: 1/40... Step: 80... Loss: 1.7767... Val Loss: 1.9205
Epoch: 1/40... Step: 90... Loss: 1.8129... Val Loss: 1.9114
Epoch: 1/40... Step: 100... Loss: 1.7887... Val Loss: 1.9194
Epoch: 1/40... Step: 110... Loss: 1.7753... Val Loss: 1.9102
Epoch: 1/40... Step: 120... Loss: 1.7621... Val Loss: 1.9096
Epoch: 1/40... Step: 130... Loss: 1.8190... Val Loss: 1.9159
Epoch: 2/40... Step: 140... Loss: 1.8436... Val Loss: 1.9283
Epoch: 2/40... Step: 150... Loss: 1.8034... Val Loss: 1.9179
Epoch: 2/40... Step: 160... Loss: 1.8296... Val Loss: 1.9112
Epoch: 2/40... Step: 170... Loss:

Epoch: 10/40... Step: 1350... Loss: 1.6650... Val Loss: 1.8235
Epoch: 10/40... Step: 1360... Loss: 1.6772... Val Loss: 1.8224
Epoch: 10/40... Step: 1370... Loss: 1.6645... Val Loss: 1.8164
Epoch: 10/40... Step: 1380... Loss: 1.7092... Val Loss: 1.8140
Epoch: 10/40... Step: 1390... Loss: 1.6977... Val Loss: 1.8139
Epoch: 11/40... Step: 1400... Loss: 1.7097... Val Loss: 1.8150
Epoch: 11/40... Step: 1410... Loss: 1.7223... Val Loss: 1.8138
Epoch: 11/40... Step: 1420... Loss: 1.6980... Val Loss: 1.8176
Epoch: 11/40... Step: 1430... Loss: 1.6792... Val Loss: 1.8192
Epoch: 11/40... Step: 1440... Loss: 1.7123... Val Loss: 1.8141
Epoch: 11/40... Step: 1450... Loss: 1.6461... Val Loss: 1.8150
Epoch: 11/40... Step: 1460... Loss: 1.6702... Val Loss: 1.8124
Epoch: 11/40... Step: 1470... Loss: 1.6558... Val Loss: 1.8087
Epoch: 11/40... Step: 1480... Loss: 1.6707... Val Loss: 1.8121
Epoch: 11/40... Step: 1490... Loss: 1.6728... Val Loss: 1.8113
Epoch: 11/40... Step: 1500... Loss: 1.6589... Val Loss:

Epoch: 20/40... Step: 2660... Loss: 1.5945... Val Loss: 1.7474
Epoch: 20/40... Step: 2670... Loss: 1.6176... Val Loss: 1.7402
Epoch: 20/40... Step: 2680... Loss: 1.6067... Val Loss: 1.7435
Epoch: 20/40... Step: 2690... Loss: 1.5962... Val Loss: 1.7463
Epoch: 20/40... Step: 2700... Loss: 1.6118... Val Loss: 1.7413
Epoch: 20/40... Step: 2710... Loss: 1.5736... Val Loss: 1.7412
Epoch: 20/40... Step: 2720... Loss: 1.5922... Val Loss: 1.7461
Epoch: 20/40... Step: 2730... Loss: 1.5821... Val Loss: 1.7477
Epoch: 20/40... Step: 2740... Loss: 1.5721... Val Loss: 1.7461
Epoch: 20/40... Step: 2750... Loss: 1.5806... Val Loss: 1.7468
Epoch: 20/40... Step: 2760... Loss: 1.5693... Val Loss: 1.7402
Epoch: 20/40... Step: 2770... Loss: 1.6036... Val Loss: 1.7365
Epoch: 20/40... Step: 2780... Loss: 1.6094... Val Loss: 1.7350
Epoch: 21/40... Step: 2790... Loss: 1.6217... Val Loss: 1.7358
Epoch: 21/40... Step: 2800... Loss: 1.6250... Val Loss: 1.7364
Epoch: 21/40... Step: 2810... Loss: 1.6172... Val Loss:

Epoch: 29/40... Step: 3970... Loss: 1.5520... Val Loss: 1.6945
Epoch: 29/40... Step: 3980... Loss: 1.5041... Val Loss: 1.6893
Epoch: 29/40... Step: 3990... Loss: 1.5256... Val Loss: 1.6834
Epoch: 29/40... Step: 4000... Loss: 1.5189... Val Loss: 1.6911
Epoch: 29/40... Step: 4010... Loss: 1.5175... Val Loss: 1.6814
Epoch: 29/40... Step: 4020... Loss: 1.5192... Val Loss: 1.6909
Epoch: 29/40... Step: 4030... Loss: 1.5256... Val Loss: 1.6832
Epoch: 30/40... Step: 4040... Loss: 1.5274... Val Loss: 1.6910
Epoch: 30/40... Step: 4050... Loss: 1.5289... Val Loss: 1.6903
Epoch: 30/40... Step: 4060... Loss: 1.5483... Val Loss: 1.6846
Epoch: 30/40... Step: 4070... Loss: 1.5361... Val Loss: 1.6806
Epoch: 30/40... Step: 4080... Loss: 1.5106... Val Loss: 1.6767
Epoch: 30/40... Step: 4090... Loss: 1.5416... Val Loss: 1.6779
Epoch: 30/40... Step: 4100... Loss: 1.5044... Val Loss: 1.6770
Epoch: 30/40... Step: 4110... Loss: 1.5243... Val Loss: 1.6796
Epoch: 30/40... Step: 4120... Loss: 1.5105... Val Loss:

Epoch: 38/40... Step: 5280... Loss: 1.4732... Val Loss: 1.6471
Epoch: 39/40... Step: 5290... Loss: 1.4848... Val Loss: 1.6450
Epoch: 39/40... Step: 5300... Loss: 1.4757... Val Loss: 1.6430
Epoch: 39/40... Step: 5310... Loss: 1.4966... Val Loss: 1.6413
Epoch: 39/40... Step: 5320... Loss: 1.5009... Val Loss: 1.6396
Epoch: 39/40... Step: 5330... Loss: 1.4548... Val Loss: 1.6374
Epoch: 39/40... Step: 5340... Loss: 1.4699... Val Loss: 1.6363
Epoch: 39/40... Step: 5350... Loss: 1.4634... Val Loss: 1.6376
Epoch: 39/40... Step: 5360... Loss: 1.5054... Val Loss: 1.6498
Epoch: 39/40... Step: 5370... Loss: 1.4537... Val Loss: 1.6449
Epoch: 39/40... Step: 5380... Loss: 1.4613... Val Loss: 1.6440
Epoch: 39/40... Step: 5390... Loss: 1.4502... Val Loss: 1.6429
Epoch: 39/40... Step: 5400... Loss: 1.4546... Val Loss: 1.6414
Epoch: 39/40... Step: 5410... Loss: 1.4628... Val Loss: 1.6436
Epoch: 39/40... Step: 5420... Loss: 1.4604... Val Loss: 1.6417
Epoch: 40/40... Step: 5430... Loss: 1.4751... Val Loss:

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `no_of_seqs_per_batch` - Number of sequences running through the network in one pass.
* `no_of_timesteps_per_sequence` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [32]:
# change the name, for saving multiple files
model_name = 'lstm_epoch.network'
checkpoint = {'no_of_hidden_nodes_in_each_lstm_cell': network.no_of_hidden_nodes_in_each_lstm_cell,
              'no_of_stacked_lstm_layers': network.no_of_stacked_lstm_layers,
              'state_dict': network.state_dict(),
              'tokens': network.character_vocabulary}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Sampling

Now that the model is trained, we'll want to sample from it. To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### Top K sampling

Our predictions come from a categorcial probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text.

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [33]:
def sample(trained_network_which_will_sample, no_of_characters_to_generate_in_the_sequence, 
           initilaization_characters_instead_of_random_start='The', top_k=None, cuda=False):
        
    if cuda:
        trained_network_which_will_sample.cuda()
    else:
        trained_network_which_will_sample.cpu()

    #work on evaluation mode    
    trained_network_which_will_sample.eval()
    
    # First off, run through the prime/ non-random characters
    characters_list = [ch for ch in initilaization_characters_instead_of_random_start]
    
    #print(characters_list)
#     import time
#     time.sleep(333)
    
    #In RNN, you learn the weights for differnet nodes, however, you do not learn the states. Hence, even during
    #evaluation, you have to initialize the hidden and cell states.
    initialization_of_hidden_state = trained_network_which_will_sample.init_hidden(1)
    
    for character in initilaization_characters_instead_of_random_start:
        character, hidden_state = trained_network_which_will_sample.predict(character, initialization_of_hidden_state, cuda=cuda, top_k=top_k)

    characters_list.append(character)
    
    # Now pass in the previous character and get a new one
    for index in range(no_of_characters_to_generate_in_the_sequence):
        
        character, hidden_state = trained_network_which_will_sample.predict(characters_list[-1], hidden_state, cuda=cuda, top_k=top_k)
        characters_list.append(character)

    return ''.join(characters_list)

In [34]:
print(sample(network, 2000, initilaization_characters_instead_of_random_start='He', top_k=5, cuda=True))

Hed how he had as the more assited times were at the talls to her, and the pace of his
husband a stall for his figures of the siren shopters, all what he treated
the saying out of the courte of the passed and thing to have becouse seen
a look as shall to a cheets and way to be a long atorather at a serfor than it
and walked out than a canneated was to saying them to shoot or him it, as
he seen it.

"Yes, I can to say, you wouldn't like that is now to the same,
that this was all to say.

"I am son in all the past in it's the comportary out of his hand all,
the day, brother in the came that which who cried the don't come of her
took."

"Oh, yes she won't in the stard." "I am not think to have this somith
to starr and a mother to the stince--hore that shatisfacte to same to
make himself indeed, he had not a conversation with whise that that
it was a great to mile to the come what see as all his sigh in the
portity, whom a serming he did not can to beer at a line. The stood, she
was stardl

## Loading a checkpoint

In [35]:
# Here we have loaded in a model that trained over 1 epoch `rnn_1_epoch.net`
with open('lstm_epoch.network', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = LSTM_Char_Generation(checkpoint['tokens'], no_of_hidden_nodes_in_each_lstm_cell=checkpoint['no_of_hidden_nodes_in_each_lstm_cell'], no_of_stacked_lstm_layers=checkpoint['no_of_stacked_lstm_layers'])

loaded.load_state_dict(checkpoint['state_dict'])


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [36]:
# Change cuda to True if you are using GPU!
print(sample(loaded, 2000, cuda=True, top_k=5, initilaization_characters_instead_of_random_start="And Levin said"))

And Levin saides. I don't have meant. They took happened, and that she was to all themether,
but it. I was driven, and stond an his mother. So this whole truing, and
when the down that the mean was nought to bread in a love take, he was
stinging over the pertom of at a musiman to brought of a gonerness, and
sometement with the provinces of his firs and the mones, as a little seemed to
bring is serelity. This is stire, better to mears when the start of the
could of at in itso and the stoom of a came from. Ther heart of has
to be and that he had been supping of the sinders, as to sated on horse
to be to been time it that he would be shoot at those that he was to that
was too half-heart, but with a letter whise sore of it....

"The marre and imaginal, I don't think to him."

"I come not is!"

And that he was somptery and hearthy and a grass all the positeds of the
pointicully shearing to them, the children to time an the same, the which
he could not trouble her fash of
thic amariag of the